In [1]:
import pandas as pd
import unidecode
import numpy as np
from src.data.utils import (
    process_food_element, treat_text, read_data, save_as_pickle, get_response_dataframe_from_dict_with_categs
)
from src.data.read_human_processed_information import x_to_dict, r_to_dict, s_to_dict, statistics_of_response_var

In [2]:
path_r = '../data/external/created_by_members/Base de datos FAI RZR.xlsx'
r_dict = r_to_dict(path_r)

path_x = '../data/external/created_by_members/Base de datos FAI XVR.xlsx'
x_dict = x_to_dict(path_x)

path_s = '../data/external/created_by_members/depuración.csv'
s_dict = s_to_dict(path_s)

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [3]:
#save_as_pickle(r_dict, '../data/data_dict/r_dict.pkl')
#save_as_pickle(x_dict, '../data/data_dict/x_dict.pkl')
#save_as_pickle(s_dict, '../data/data_dict/s_dict.pkl')

In [6]:
response_r = get_response_dataframe_from_dict_with_categs(r_dict)

In [7]:
response_x = get_response_dataframe_from_dict_with_categs(x_dict)

In [8]:
response_s = get_response_dataframe_from_dict_with_categs(s_dict)

In [9]:
statistics_of_response_var(response_x)

{'target_proportion': 0.013996393619574023,
 'instances_number': 31680518,
 'nunique_foods_considered': 5623,
 'unique_food_considered': array([2340762, 2340769, 2340771, ..., 2346381, 2346382, 2346383])}

In [10]:
statistics_of_response_var(response_r)

{'target_proportion': 0.0402689249411127,
 'instances_number': 5586094,
 'nunique_foods_considered': 2362,
 'unique_food_considered': array([2341298, 2345702, 2345722, ..., 2342919, 2340844, 2340845])}

In [11]:
statistics_of_response_var(response_s)

{'target_proportion': 0.32694682310520445,
 'instances_number': 31629376,
 'nunique_foods_considered': 5624,
 'unique_food_considered': array([2341335, 2341336, 2341337, ..., 2345829, 2345830, 2345831])}

In [17]:
######
data_path = '../data/raw/FoodData_Central_csv_2023-04-20/'

food = read_data(f'{data_path}food.csv')
survey_fndds_food = read_data(f'{data_path}survey_fndds_food.csv')
nutrient_data = read_data(f'{data_path}food_nutrient.csv')
nutrient_info = read_data(f'{data_path}nutrient.csv')
input_food = read_data(f'{data_path}input_food.csv')
fndds_ingredient_nutrient_value = read_data(f'{data_path}fndds_ingredient_nutrient_value.csv')
fndds_derivation = read_data(f'{data_path}fndds_derivation.csv')
#category = pd.read_csv(f'{data_path}food_category.csv')
component = read_data(f'{data_path}food_component.csv')
category = read_data(f'{data_path}wweia_food_category.csv')


nutrient_info.loc[
    nutrient_info.nutrient_nbr.notna(), 'nutrient_nbr_int'
] = nutrient_info.loc[nutrient_info.nutrient_nbr.notna()].nutrient_nbr.astype('int')


fndss_dt = food.data_type == 'survey_fndds_food'
relevant_list = ['description', 'fdc_id', 'food_category_id', 'publication_date']
food_survey_food = survey_fndds_food.merge(food[fndss_dt][relevant_list], on='fdc_id', how='left')

food_survey_food = food_survey_food\
.merge(category, how='left', left_on='wweia_category_code', right_on='wweia_food_category')\
.drop(['wweia_category_code', 'food_category_id', 'start_date', 'end_date', 'publication_date'], axis=1)



relevant_indexes = statistics_of_response_var(response_r)['unique_food_considered']
food_ingredient_nutrition_info = {}
for food in relevant_indexes:
    food_ingredient_nutrition_info[food] = {}
    ingredients, missing_ingredients, nutrition = get_food_information(food)
    food_ingredient_nutrition_info[food]['ingredients'] = ingredients[['sr_code', 'sr_description']].to_dict()
    food_ingredient_nutrition_info[food]['missing_ingredients'] = missing_ingredients.sr_code.to_dict()
    food_ingredient_nutrition_info[food]['nutrition'] = nutrition.to_dict()
    food_ingredient_nutrition_info[food]['unique_ingredients_code'] = ingredients.sr_code.unique()
    food_ingredient_nutrition_info[food]['unique_ingredients'] = ingredients.sr_description.unique()

/mnt/c/Users/dhdzm/Documents/projects/ingeineria_en_alimentos/ingenieria_en_alimentos/ai_in_food/src/data/utils.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path, **additional_kwargs)


In [20]:
food_ingredient_nutrition_info_processed = pd.DataFrame(food_ingredient_nutrition_info).T
food_ingredient_nutrition_info_processed.to_csv('../data/interim/food_ingredient_nutrition_info_processed.csv')

In [21]:
def get_unique_nutrition_names(info):
    info = pd.DataFrame(info)
    return list(info.name.unique())
    
all_available_nutrients = sorted(list(set(food_ingredient_nutrition_info_processed.nutrition.apply(get_unique_nutrition_names).sum())))

In [52]:
saving_all_info = []
for i in range(food_ingredient_nutrition_info_processed.shape[0]):
    test = pd.DataFrame(food_ingredient_nutrition_info_processed.nutrition.iloc[i])
    test_idx = food_ingredient_nutrition_info_processed.iloc[i].name
    missing_nutrients = set(all_available_nutrients) - set(test.name.to_list())
    test_n = test.groupby(['name'])[['Nutrient value']].sum()
    test_mn = pd.DataFrame(missing_nutrients, columns=['name'])
    test_mn['Nutrient value'] = np.nan
    test_mn.set_index('name', inplace=True)
    test = pd.concat([test_n, test_mn]).sort_index().T
    test.index = [test_idx]
    saving_all_info.append(test)

In [55]:
saving_all_info_df = pd.concat(saving_all_info).reset_index().rename(columns={'index':'fdc_id'})

In [56]:
saving_all_info_df

name,fdc_id,Caffeine,"Calcium, Ca","Carbohydrate, by difference","Carotene, alpha","Carotene, beta",Cholesterol,"Choline, total","Copper, Cu","Cryptoxanthin, beta",...,"Vitamin B-12, added",Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),Vitamin E (alpha-tocopherol),"Vitamin E, added",Vitamin K (phylloquinone),Water,Zeaxanthin,"Zinc, Zn"
0,2341298,NaN,72.0,NaN,NaN,NaN,174.0,82.6,0.090,NaN,...,NaN,0.192,NaN,4.3,0.60,NaN,NaN,26.68,NaN,0.86
1,2345702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2345722,NaN,69.0,NaN,NaN,NaN,472.0,392.8,0.040,NaN,...,NaN,0.140,NaN,15.0,6.80,NaN,3.1,19.40,NaN,1.46
3,2345723,NaN,NaN,NaN,NaN,NaN,95.0,49.7,NaN,NaN,...,NaN,NaN,NaN,2.5,0.60,NaN,NaN,NaN,NaN,0.11
4,2343975,NaN,643.0,69.40,NaN,NaN,NaN,45.4,0.467,NaN,...,3.57,0.714,NaN,7.1,5.00,4.87,2.3,6.80,170.0,3.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,2340821,NaN,99.0,10.37,NaN,NaN,3.0,15.2,0.018,NaN,...,NaN,0.050,NaN,0.9,0.02,NaN,0.1,80.26,NaN,0.46
2358,2340822,NaN,100.0,9.54,NaN,3.0,5.0,33.4,0.018,NaN,...,NaN,0.050,NaN,0.9,0.02,NaN,0.2,78.76,NaN,0.46
2359,2342919,NaN,118.0,15.96,NaN,20.0,NaN,48.4,0.075,NaN,...,NaN,0.020,2.5,NaN,0.31,NaN,3.5,77.50,NaN,0.31
2360,2340844,NaN,121.0,4.66,NaN,5.0,13.0,15.2,0.009,NaN,...,NaN,0.032,0.5,10000.1,0.06,NaN,0.2,87.90,NaN,0.59


In [12]:
a = response_r\
.merge(saving_all_info_df, on='fdc_id', how='inner')\
.merge(saving_all_info_df, left_on='fdc_id_to', right_on='fdc_id', how='inner')

NameError: name 'saving_all_info_df' is not defined

In [14]:
a

NameError: name 'a' is not defined